In [1]:
import os
import csv
from bs4 import BeautifulSoup
import pandas as pd
import requests as rq
import time
import random
import unicodedata
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import *
import string

In [2]:
path = os.getcwd()
movies = pd.DataFrame(pd.read_html(path + "\\movies-1.html")[0])
movies.drop('Id', inplace=True, axis = 1)

In [3]:
# for i in range(len(movies)):
#     try:
#         response = rq.get(movies.URL[i])
#     except rq.exceptions.RequestException as e:
#         print(e)
#         time.sleep(20*60 + 30)
#         response = rq.get(movies.URL[i])
#     soup = BeautifulSoup(response.text, 'html.parser')
#     f = open('article_'+str(i)+'.html','w')
#     f.write(str(soup))
#     f.close()
#     time.sleep(random.choice(range(1,6)))

In [4]:
path1 = path + '\\Articles'

In [103]:
for i in range(0, 20):
    article = open(path1+'\\article_'+str(i)+'.html', 'r')
    soup = BeautifulSoup(article, 'html.parser')
    d = {}
    try:
        for x in soup.find('table', class_="infobox vevent").find('th').find_all_next('th'):
            d[x.text] = unicodedata.normalize('NFKD',x.next_sibling.get_text(separator = '<br/>').replace('<br/>', ','))
    except:
        pass
    title = str(soup.select('h1')[0].text)
    #the title had "- wikipedia" after the film name. to remove "- wikipedia", I used split function and then took the first part
    
    start = soup.find('p')
    intro = start.text.strip()
    for elem in start.next_siblings:
        if elem.name != 'p':
            break
        intro += elem.text.strip()   
          
    try:
        start = soup.find('h2').find_next('p')
        plot = start.text.strip()
        for elem in start.next_siblings:
            if elem.name != 'p':
                break
            plot += elem.text.strip()
        
    except:
        plot = "NA"
    

        
        
    try :
            director = d['Directed by']
    except: 
            director = "NA"
            
    try :
            producer = d['Produced by']
    except: 
            producer = "NA"
            
    try :
        
            writer = d["Written by"]
    except:
            writer = "NA"
    
    
    try :
            starring = d["Starring"].strip()
    
    except:
            starring = "NA"
            
    try :
        
            music = d["Music by"]
    except :
            music = "NA"
        
    try :
            date = d["Release date"]
            release_date = date[date.find("("):date.find(")")][2:]
            
    except :
        
            release_date = "NA"
            
    try :
            run_time = d["Running time"]
            
    except :
            run_time = "NA"
            
    try :
            country = d["Country"]
            
    except :
            
            country = "NA"
            
    try :
            language = d["Language"]
            
    except:
            language = "NA"
            
            
    try :
            budget = d["Budget"]
            
    except :
            budget = "NA"
            

    with open(path+"\\TSV\\article_" + str(i) + ".tsv", "w" ,encoding="utf-8") as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow([title, intro, plot, director, producer, writer , starring, music, release_date,run_time, country , language , budget])


In [108]:
soup.find('table', class_="infobox vevent").find('th').find_all_next('th')

[<th scope="row" style="white-space:nowrap;padding-right:0.65em;">Written by</th>,
 <th scope="row" style="white-space:nowrap;padding-right:0.65em;">Based on</th>,
 <th scope="row" style="white-space:nowrap;padding-right:0.65em;">Starring</th>,
 <th scope="row" style="white-space:nowrap;padding-right:0.65em;">Distributed by</th>,
 <th scope="row" style="white-space:nowrap;padding-right:0.65em;"><div style="padding:0.1em 0;line-height:1.2em;white-space:normal;">Release date</div></th>,
 <th scope="row" style="white-space:nowrap;padding-right:0.65em;"><div style="padding:0.1em 0;line-height:1.2em;white-space:normal;">Running time</div></th>,
 <th scope="row" style="white-space:nowrap;padding-right:0.65em;">Country</th>,
 <th scope="row" style="white-space:nowrap;padding-right:0.65em;">Language</th>,
 <th class="navbox-title" colspan="2" scope="col"><div class="plainlinks hlist navbar mini"><ul><li class="nv-view"><a href="/wiki/Template:A_Christmas_Carol" title="Template:A Christmas Caro

In [9]:
stop_words = set(stopwords.words('english')) 
stemmer = PorterStemmer()

In [190]:
for i in range(0,20):
    f = open("articl_" + str(i) + ".tsv", encoding = "utf-8") 
    text = f.read()
    f.close()
    
    words = word_tokenize(text) #devide the text into substrings
    filtered1 = [w for w in words if not w in stop_words] #remove stop words
    filtered2 = list(filter(lambda word: word not in string.punctuation, filtered1)) #remove punctuation
    filtered3 = [stemmer.stem(w) for w in filtered2] #stemming
    filtered4 = [c.replace("''", "").replace("``", "") for c in filtered3 ] #removing useless '' and  `` characters

with open(path + "\\WORDS\\final_" + str(i) + ".tsv", "w" ,encoding="utf-8") as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow(filtered4)

